<a href="https://colab.research.google.com/github/srinidhibv1403/1BM23CS339-BIS/blob/main/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import math  # For gamma function

# Objective function (example: Sphere function)
def objective_function(x):
    return np.sum(x**2)

# Lévy flight function to generate step sizes
def levy_flight(Lambda, dim):
    sigma_u = (math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
              (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.normal(0, sigma_u, size=dim)
    v = np.random.normal(0, 1, size=dim)
    step = u / (np.abs(v) ** (1 / Lambda))
    return step

def cuckoo_search(n=15, dim=2, pa=0.25, max_gen=100):
    # Initialize nests randomly in the range [-10, 10]
    nests = np.random.uniform(-10, 10, size=(n, dim))
    fitness = np.array([objective_function(nest) for nest in nests])

    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx].copy()
    best_fitness = fitness[best_idx]

    alpha = 0.01  # Step size scaling factor
    Lambda = 1.5  # Lévy flight parameter

    for gen in range(max_gen):
        for i in range(n):
            # Generate new solution by Lévy flight
            step = levy_flight(Lambda, dim)
            new_nest = nests[i] + alpha * step

            # Keep new solution within bounds [-10, 10]
            new_nest = np.clip(new_nest, -10, 10)

            f_new = objective_function(new_nest)

            # Choose a random nest j to potentially replace
            j = np.random.randint(n)
            if f_new < fitness[j]:
                nests[j] = new_nest
                fitness[j] = f_new

                # Update best solution found so far
                if f_new < best_fitness:
                    best_fitness = f_new
                    best_nest = new_nest.copy()

        # Abandon a fraction pa of worst nests and replace with new random ones
        num_abandon = int(pa * n)
        worst_indices = np.argsort(fitness)[-num_abandon:]
        nests[worst_indices] = np.random.uniform(-10, 10, size=(num_abandon, dim))
        fitness[worst_indices] = np.array([objective_function(nest) for nest in nests[worst_indices]])

        # Update best solution after abandonment
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_fitness:
            best_fitness = fitness[current_best_idx]
            best_nest = nests[current_best_idx].copy()

    return best_nest, best_fitness

# Run example
if __name__ == "__main__":
    best_solution, best_value = cuckoo_search()
    print("Best solution found:", best_solution)
    print("Best objective value:", best_value)


Best solution found: [-1.81712676e-05  1.45007457e-04]
Best objective value: 2.1357357438781935e-08
